In [0]:
%sql
-- KPI 1: Total number of customers processed by the trust scoring pipeline
SELECT COUNT(*) AS total_customers
FROM ai_trust_score_project.trust_predictions_final;

In [0]:
%sql
-- KPI 2: Count of customers by final risk decision
SELECT 
  final_decision,
  COUNT(*) AS customer_count
FROM ai_trust_score_project.trust_predictions_final
GROUP BY final_decision;

In [0]:
%sql
-- KPI 3: Percentage of customers classified as HIGH_RISK
SELECT 
  ROUND(
    100.0 * SUM(CASE WHEN final_decision = 'HIGH_RISK' THEN 1 ELSE 0 END) / COUNT(*),
    2
  ) AS high_risk_percentage
FROM ai_trust_score_project.trust_predictions_final;

In [0]:
%sql
-- KPI 4: Overall average prediction confidence (model certainty)
SELECT 
  ROUND(AVG(prediction_confidence), 3) AS avg_prediction_confidence
FROM ai_trust_score_project.trust_predictions_final;

In [0]:
%sql
-- KPI 5: Average confidence for HIGH_RISK vs LOW_RISK customers
SELECT 
  final_decision,
  ROUND(AVG(prediction_confidence), 3) AS avg_confidence
FROM ai_trust_score_project.trust_predictions_final
GROUP BY final_decision;

In [0]:
%sql
-- KPI 6: Distribution of customers by confidence buckets
SELECT
  CASE
    WHEN prediction_confidence >= 0.9 THEN '0.9 - 1.0'
    WHEN prediction_confidence >= 0.7 THEN '0.7 - 0.9'
    WHEN prediction_confidence >= 0.5 THEN '0.5 - 0.7'
    ELSE '< 0.5'
  END AS confidence_bucket,
  COUNT(*) AS customers
FROM ai_trust_score_project.trust_predictions_final
GROUP BY confidence_bucket
ORDER BY confidence_bucket DESC;

In [0]:
%sql
-- KPI 7: Top 20 customers with highest churn risk (actionable list)
SELECT 
  customerid,
  prediction_confidence
FROM ai_trust_score_project.trust_predictions_final
ORDER BY prediction_confidence DESC
LIMIT 20;

In [0]:
%sql
-- KPI 8: Customers where the model is least confident
SELECT 
  customerid,
  prediction_confidence
FROM ai_trust_score_project.trust_predictions_final
ORDER BY prediction_confidence ASC
LIMIT 20;

In [0]:
%sql
-- KPI 9: Expected number of churners (sum of probabilities)
SELECT 
  ROUND(SUM(prediction_confidence), 2) AS expected_churn_customers
FROM ai_trust_score_project.trust_predictions_final;

In [0]:
%sql
-- KPI 10: Standard deviation of confidence scores (model stability indicator)
SELECT 
  ROUND(STDDEV(prediction_confidence), 3) AS confidence_std_dev
FROM ai_trust_score_project.trust_predictions_final;

In [0]:
%sql
-- KPI 11: Average churn risk by subscription type
SELECT 
  c.subscription_type,
  COUNT(*) AS customers,
  ROUND(AVG(p.prediction_confidence), 3) AS avg_risk
FROM ai_trust_catalog.churn_trust.silver_customer_churn c
JOIN ai_trust_score_project.trust_predictions_final p
ON c.customerid = p.customerid
GROUP BY c.subscription_type
ORDER BY avg_risk DESC;

In [0]:
%sql
-- KPI 12: Average risk across age segments
SELECT
  CASE
    WHEN age < 25 THEN 'Under 25'
    WHEN age BETWEEN 25 AND 40 THEN '25-40'
    WHEN age BETWEEN 41 AND 60 THEN '41-60'
    ELSE '60+'
  END AS age_group,
  ROUND(AVG(p.prediction_confidence), 3) AS avg_risk
FROM ai_trust_catalog.churn_trust.silver_customer_churn c
JOIN ai_trust_score_project.trust_predictions_final p
ON c.customerid = p.customerid
GROUP BY age_group
ORDER BY avg_risk DESC;

In [0]:
%sql
-- KPI 13: Churn risk based on customer tenure
SELECT
  CASE
    WHEN tenure < 1 THEN '< 1 year'
    WHEN tenure BETWEEN 1 AND 3 THEN '1-3 years'
    WHEN tenure BETWEEN 4 AND 6 THEN '4-6 years'
    ELSE '6+ years'
  END AS tenure_bucket,
  ROUND(AVG(p.prediction_confidence), 3) AS avg_risk
FROM ai_trust_catalog.churn_trust.silver_customer_churn c
JOIN ai_trust_score_project.trust_predictions_final p
ON c.customerid = p.customerid
GROUP BY tenure_bucket
ORDER BY avg_risk DESC;

In [0]:
%sql
-- KPI 14: High-risk customers who spend the most (revenue exposure)
SELECT 
  c.customerid,
  c.total_spend,
  p.prediction_confidence
FROM ai_trust_catalog.churn_trust.silver_customer_churn c
JOIN ai_trust_score_project.trust_predictions_final p
ON c.customerid = p.customerid
WHERE p.final_decision = 'HIGH_RISK'
ORDER BY c.total_spend DESC
LIMIT 20;

In [0]:
%sql
-- KPI 15: Relationship between support calls and churn risk
SELECT
  support_calls,
  ROUND(AVG(p.prediction_confidence), 3) AS avg_risk
FROM ai_trust_catalog.churn_trust.silver_customer_churn c
JOIN ai_trust_score_project.trust_predictions_final p
ON c.customerid = p.customerid
GROUP BY support_calls
ORDER BY support_calls;

In [0]:
%sql
-- KPI 16: Impact of payment delays on churn risk
SELECT
  payment_delay,
  ROUND(AVG(p.prediction_confidence), 3) AS avg_risk
FROM ai_trust_catalog.churn_trust.silver_customer_churn c
JOIN ai_trust_score_project.trust_predictions_final p
ON c.customerid = p.customerid
GROUP BY payment_delay
ORDER BY payment_delay;

In [0]:
%sql
-- KPI 17: Average churn risk by gender
SELECT
  gender,
  COUNT(*) AS customers,
  ROUND(AVG(p.prediction_confidence), 3) AS avg_risk
FROM ai_trust_catalog.churn_trust.silver_customer_churn c
JOIN ai_trust_score_project.trust_predictions_final p
ON c.customerid = p.customerid
GROUP BY gender
ORDER BY avg_risk DESC;

In [0]:
%sql
-- KPI 18: High-risk customers with low usage frequency
SELECT
  c.customerid,
  c.usage_frequency,
  p.prediction_confidence
FROM ai_trust_catalog.churn_trust.silver_customer_churn c
JOIN ai_trust_score_project.trust_predictions_final p
ON c.customerid = p.customerid
WHERE p.final_decision = 'HIGH_RISK'
ORDER BY c.usage_frequency ASC
LIMIT 10;

In [0]:
%sql
-- KPI 19: High-risk customers with longest inactivity
SELECT
  c.customerid,
  c.last_interaction,
  p.prediction_confidence
FROM ai_trust_catalog.churn_trust.silver_customer_churn c
JOIN ai_trust_score_project.trust_predictions_final p
ON c.customerid = p.customerid
WHERE p.final_decision = 'HIGH_RISK'
ORDER BY c.last_interaction DESC
LIMIT 10;

In [0]:
%sql
-- KPI 20: Executive summary of risk exposure
SELECT
  COUNT(*) AS total_customers,
  SUM(CASE WHEN final_decision = 'HIGH_RISK' THEN 1 ELSE 0 END) AS high_risk_customers,
  ROUND(AVG(prediction_confidence), 3) AS avg_risk_score
FROM ai_trust_score_project.trust_predictions_final;